In [13]:
# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)
# cifar10 data - 32x32
from tensorflow.keras.datasets import cifar10

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os


import random

- LOad images in with specific size 
- create modal with input size 
- maybe normalise images 
- binarize labels.

# Json FIles

In [2]:
test_metadata = pd.read_json(os.path.join("..", "..", "..", "431824", "images", "metadata", "test_data.json"), lines=True) # Loading in JSON files of metadata
train_metadata = pd.read_json(os.path.join("..", "..", "..", "431824", "images", "metadata", "train_data.json"), lines=True)
val_metadata = pd.read_json(os.path.join("..", "..", "..", "431824", "images", "metadata", "val_data.json"), lines=True)

In [ ]:
def data_sampling(comments_list):
    print("Creating Sampel")
    thousand_comments = random.sample(comments_list, 10) # Taking 1000 random comments from the list.
    print("Sample size: " + str(len(thousand_comments))) # Printing sample length
    return thousand_comments

In [16]:
test_metadata = test_metadata.sample(3000)
train_metadata = train_metadata.sample(10000)
val_metadata = val_metadata.sample(2000)

In [24]:
val_metadata

,image_url,image_path,brand,product_title,class_label,color
1007,https://m.media-amazon.com/images/I/71Xt86p9vJ...,images/val/1007.jpeg,Buy That Trendz,Combo Offer (Pack of 3) Cotton Viscose Lycra D...,dhoti_pants,NaN
4084,https://m.media-amazon.com/images/I/51qMnlhTRl...,images/val/4084.jpeg,Harshil,Shahi StyleVelvet Mojari Jutti Ethnic Casual S...,mojaris_men,NaN
4257,https://rukminim1.flixcart.com/image/309/371/k...,images/val/4257.jpeg,BELJER,Men's stylish and comfortable shoes | Flaxible...,mojaris_men,N/A
4794,https://m.media-amazon.com/images/I/31ivqCDd0M...,images/val/4794.jpeg,Rajwari,Black Tan Kohlapuri Chappal,mojaris_women,NaN
5446,https://m.media-amazon.com/images/I/61OH25yQLO...,images/val/5446.jpeg,Mulberry Lifestyle,Men Mandarin Collar Jacket,nehru_jackets,NaN
...,...,...,...,...,...,...
2694,https://m.media-amazon.com/images/I/71pv58lakr...,images/val/2694.jpeg,Integriti,Men's Cotton pathan Suit Kurta (PTN-SK-5015_Wh...,kurta_men,NaN
335,https://m.media-amazon.com/images/I/61l1ns1H8F...,images/val/335.jpeg,SBC,Shree Bajarang Creation Women's Silk Saree wit...,saree,NaN
2150,https://m.media-amazon.com/images/I/71GRlpcRpv...,images/val/2150.jpeg,W for Woman,Women's Regular Kurta,gowns,NaN
4741,https://rukminim1.flixcart.com/image/309/371/j...,images/val/4741.jpeg,TAMANNA,Ethnic handmade leather khussa jutti Bellies F...,mojaris_women,Multicolor


In [19]:
train_metadata["class_label"].unique()

array(['dupattas', 'women_kurta', 'sherwanis', 'gowns', 'nehru_jackets',
       'saree', 'mojaris_women', 'blouse', 'leggings_and_salwars',
       'kurta_men', 'petticoats', 'lehenga', 'mojaris_men', 'palazzos',
       'dhoti_pants'], dtype=object)

## Code with flow from dir

In [4]:

# Data augmentaion 
# ImageDataGenerator from tensorflow 
datagen = ImageDataGenerator(horizontal_flip=True, # Flip it horizontally around the access
                             rotation_range=20, # Rotate the image randomly 20 degress around the access
                             rescale = 1/255 # rescale it between 0-1
)
# Take your images, create a pipelie (Take an image modify it, pass it on)

In [5]:
directory_images = os.path.join("..","..","..", "431824") # Defining path to images, the rest of the path is defined in the metadata


In [20]:
X_train_tf = datagen.flow_from_dataframe( # using keras flow  from dataframe 
    dataframe = train_metadata, # Defining dataframe 
    directory = directory_images, # Path to images 
    x_col = "image_path", # rest of the image path from dataframe 
    y_col = "class_label", # column label
    subset = "training", # what this data is 
    target_size=(224, 224), # image should be loaded in as size 
    color_mode="rgb", # colors 
    class_mode = "categorical", # One hot encoding the labels 
    batch_size = 128, # take images of batchs 128 at a time
    shuffle = True # shuffle the images around 
)

Found 10000 validated image filenames belonging to 15 classes.


In [21]:
val_tf = datagen.flow_from_dataframe(
    dataframe = val_metadata,
    directory = directory_images,
    x_col = "image_path",
    y_col = "class_label",
    #subset = "validation",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode = "categorical",
    batch_size = 128,
    shuffle = True
)

Found 2000 validated image filenames belonging to 15 classes.


In [9]:
test_datagen = ImageDataGenerator(
                                rescale = 1./255. # datagenerator for test, it only has to rescale the images 
)

In [22]:
test_tf = test_datagen.flow_from_dataframe(
    dataframe = test_metadata,
    directory = directory_images,
    x_col = "image_path",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode = None,
    batch_size = 128,
    shuffle = False # do not shuffle the images 
)

Found 3000 validated image filenames.


# Model

In [11]:
# load model without classifier layers
model = VGG16(include_top=False, 
              pooling='avg',
              input_shape=(224, 224, 3))

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
    
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
bn = BatchNormalization()(flat1) # Added batnormalization from tensorflow. Take the previouslayer, normalise the values, and than pass them on
class1 = Dense(256, 
               activation='relu')(bn) # Added new classification layer 
class2 = Dense(128, 
               activation='relu')(class1)
output = Dense(15, # 15 labels
               activation='softmax')(class2)

# define new model
model = Model(inputs=model.inputs, 
              outputs=output)

# compile
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# summarize
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Train model

In [23]:
# fit the data generator to our images

# fits the model on batches with real-time data augmentation:

H = model.fit( # fitting the model to 
    X_train_tf, # training tensorflow dataframe 
    steps_per_epoch = len(X_train_tf), # take as many steps as the length of the dataframe 
    validation_data = val_tf, # Validation data 
    validation_steps = len(val_tf), 
    epochs = 10
)
# Possible to get image (stream the image) (stream it frow the dataGenerator) from the folder, instead of loading the image into the script. 

Epoch 1/10


2023-04-21 14:29:54.047683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


 2/79 [..............................] - ETA: 31:56 - loss: 2.8322 - accuracy: 0.0625

KeyboardInterrupt: 